### This Notebook shows how to use a regular tensorflow model file and a tflite file to predict one image.

In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
from skimage import transform

## Load Keras Model to predict one image

Load model from file.

In [2]:
model = tf.keras.models.load_model('predict_melanoma.h5')

Load image and transform to required input format for the model.

In [3]:
img = Image.open("test_mole.jpeg")
np_image = np.array(img).astype('float32')/255
#np_image = transform.resize(np_image, (224, 224, 3))
np_image = np.expand_dims(np_image, axis=0)

Run the model prediction.

In [4]:
prediction_probabilities = model.predict(np_image)

Show class probablities [melanoma, non-melanoma]

In [5]:
print(prediction_probabilities)

[[0.7292735 0.2707265]]


Using argmax means in this example the threshold is 0.5.

In [6]:
prediction_keras = prediction_probabilities.argmax(axis=1)
print(prediction_keras)

[0]


## Load TFLite model to predict one image
https://www.tensorflow.org/lite/guide/inference

Load TFLite model and allocate tensors.

In [7]:
interpreter = tf.lite.Interpreter(model_path="../frontend_molileo/assets/predict_melanoma.tflite")
interpreter.allocate_tensors()

Get input and output tensors

In [8]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [9]:
input_details[0]['shape']

array([  1, 224, 224,   3])

Load image.

In [10]:
input_shape = input_details[0]['shape']
img = Image.open("test_mole.jpeg")

Adjust input image to the model's input shape.

In [11]:
input_data = np.array(img).astype('float32')/255
input_data = np.expand_dims(input_data, axis=0)
input_data.shape

(1, 224, 224, 3)

Run interpreter to get prediction.

In [12]:
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

Show class probablities [melanoma, non-melanoma]

In [13]:
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.7292735  0.27072647]]


Using argmax means in this example the threshold is 0.5.

In [14]:
prediction_tflite = output_data.argmax(axis=1)
print(prediction_tflite)

[0]
